In [33]:
import numpy as np
import common
import custom_library

In [34]:
class RNN :
    def __init__(self, Wx, Wh, b) :
        self.params = [Wx,Wh,b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev) :
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next) :
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis = 0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(x.T, dt)
        dWx = np.matmul(dt, Wx.T)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

In [35]:
class TimeRNN : ##RNN들을 모아놓은것
    def __init__(self, Wx, Wh, b, stateful = False) :
        self.params = [Wx, Wh, b] #가중치, 편차 저장
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] #미분값 저장
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def set_state(self, h):
        self.h = h
    def reset_state(self) :
        self.h = None

    def forward(self, xs) : #xs는 x들
        Wx, Wh, b = self.params
        N, T, D = xs.shape # N은 case, T는 시계열 수, D는 뉴런
        D, H = Wx.shape #D는 Wx의 행의크기, H는 Wx의 열의 크기
        self.layers = []
        hs = np.empty((N, T, H), dtype = 'f')

        if (not self.stateful) or (self.h is None) :
            self.h = np.zeros((N, H), dtype = 'f')
        for t in range(T) :
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:,t,:], self.h)
            hs[:,t,:] = self.h
            self.layers.append(layer)

        return hs


    def backward(self, dhs) :
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype = 'f')
        dh = 0
        grads = [0,0,0]
        for t in reversed(range(T)) :
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:,t,:] + dh)
            dxs[:,t,:] = dx

            for i, grad in enumerate(grads) :
                self.grads[i][...] = grad
            self.dh = dh

        return dxs

In [36]:
class SimpleRnnlm :
    def __init__(self, vocab_size, wordvec_size, hidden_size) :
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        
        rnn_b = np,zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.layers = [
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful = True)
        ]
        self.loss_layer = common.TimeSoftmaxWithLoss()
        self.rnn_layer = self.layer[0]

        self.params, self.grads = [], []
        for layer in self.layers :
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts) :
        for layer in self.layers :
            xs = layer.forward(xs)
        loss = self.loss_layer_forward(xs,ts)
        return loss

    def backward(self, dout = 1) :
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers) :
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()

In [41]:
batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 0
lr = 0.1

max_epoch = 100

with open("ptb.test.txt", mode = "r") as f :
    text = f.read()

corpus,id_to_word,word_to_id = custom_library.preprocess(text)

corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)
print("말뭉치 크기 : {}, 어휘 수 : {}".format(corpus_size, vocab_size))




말뭉치 크기 : 1000, 어휘 수 : 387
